In [ ]:
# !pip install xarray
# !pip install datacube
# !pip install --extra-index-url="https://packages.dea.ga.gov.au" \
#   odc-algo

In [1]:
import os
import xarray as xr
import numpy as np
import datacube
import pandas as pd
from odc.algo import xr_reproject
from datacube.utils.geometry import assign_crs
from datacube.utils.dask import start_local_dask
# from dea_tools.dask import create_local_dask_cluster

In [2]:
client = start_local_dask(mem_safety_margin='2Gb')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 1
Total threads: 16,Total memory: 44.92 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43679,Workers: 1
Dashboard: /proxy/8787/status,Total threads: 16
Started: Just now,Total memory: 44.92 GiB
Comm: tcp://127.0.0.1:35739,Total threads: 16
Dashboard: /proxy/35473/status,Memory: 44.92 GiB
Nanny: tcp://127.0.0.1:42619,


## Analysis Parameters

In [3]:
time=('2018')

## Check units of flux tower data

In [4]:
df = xr.open_dataset('https://dap.tern.org.au/thredds/dodsC/ecosystem_process/ozflux/Tumbarumba/2021_v1/L6/default/Tumbarumba_L6_20020107_20191231_Monthly.nc')

print('Ta:',df.Ta.units)
print('Sws:',df.Sws.units)
print('Precip:',df.Precip.units)
print('Fn:',df.Fn.units)

Ta: degC
Sws: m^3/m^3
Precip: mm/month
Fn: W/m^2


## Retrieving ANU climate data (1 km resolution)

Solar radiation is in units that are difficult to resolve with flux tower data

In [5]:
Ta = xr.open_dataset('https://dapds00.nci.org.au/thredds/dodsC/gh70/ANUClimate/v2-0/stable/month/tavg',
                     chunks={'time':'100mb'}).sel(time=time).compute()
precip = xr.open_dataset('https://dapds00.nci.org.au/thredds/dodsC/gh70/ANUClimate/v2-0/stable/month/rain',
                         chunks={'time':'100mb'}).sel(time=time).compute()
fn = xr.open_dataset('https://dapds00.nci.org.au/thredds/dodsC/gh70/ANUClimate/v2-0/stable/month/srad',
                     chunks={'time':'100mb'}).sel(time=time).compute()
vpd = xr.open_dataset('https://dapds00.nci.org.au/thredds/dodsC/gh70/ANUClimate/v2-0/stable/month/vpd',
                      chunks={'time':'100mb'}).sel(time=time).compute()

## Retrieving MODIS LAI (500m resolution)

load directly from g/data as url exceeds data limit.  Need permissions first

In [6]:
lai = xr.open_dataset('/g/data/ub8/au/MODIS/mosaic/MOD15A2H.006/MOD15A2H.006.b02.500m_lai.'+time+'.nc',
                      chunks=dict(latitude=1000, longitude=1000))

In [7]:
lai = lai['500m_lai'].resample(time='MS').mean().compute()

## Retrieving soil moisture (GRAFS)

load directly from g/data as url exceeds data limit.  Need permissions first

In [8]:
sws = xr.open_dataset('/g/data/ub8/global/GRAFS/GRAFS_RootzoneSoilWaterIndex_'+time+'.nc',
                      chunks=dict(latitude=1000, longitude=1000))

In [9]:
sws = sws.resample(time='MS').mean().compute()

## Reproject and merge variables

In [10]:
sws = xr_reproject(assign_crs(sws.soil_water_index, crs='epsg:4326'),
                   assign_crs(Ta, crs='epsg:4326').geobox,
                   resampling="bilinear")

In [11]:
lai = xr_reproject(assign_crs(lai, crs='epsg:4326'),
                   assign_crs(Ta, crs='epsg:4326').geobox,
                   resampling="average")

In [ ]:
#merge results into single dataset
sws = sws.rename({"longitude": "lon", "latitude": "lat"})
lai = lai.rename({"longitude": "lon", "latitude": "lat"})

result = xr.merge([sws, lai, Ta, precip, fn, vpd], compat='override').drop('crs')

In [ ]:
result